In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
from random import randint
from time import sleep
from datetime import date
from itertools import cycle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pickle
import os

In [ ]:
def start_browsing(headless=False):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-proxy-server')
    chrome_options.headless=headless
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15'
    chrome_options.add_argument('--user-agent=%s' % user_agent)
    path = '/Users/macbook/Downloads/Chrome Driver'
    browser =  webdriver.Chrome(os.path.join(path, 'chromedriver'),chrome_options=chrome_options)
    browser.maximize_window()
    return browser

### Main functions to parse a product page

In [ ]:
def get_option_links():
    path = '//*[@id="product-information"]//a'
    opt_links = [x.get_attribute('href') for x in browser.find_elements_by_xpath(path)]
    
    path = '//*[contains(@class, "dropdown_select")]//option'
    opt_links.extend([x.get_attribute('value') for x in browser.find_elements_by_xpath(path)])
    
    return opt_links

In [ ]:
def process_temp_links(temp_links):
    clean_links = []
    for link in temp_links:
        if 'https://www.hornbach.de' not in link:
            link = 'https://www.hornbach.de' + link
        if '#attribute' not in link and link not in clean_links:
            clean_links.append(link)
    return  clean_links

In [ ]:
def collect_links(links):
    for link in links:
        if link not in all_links:
            all_links.append(link)    

In [ ]:
def get_img_link():
    # Click on Image to get large Image
    open_close_big_img()
    
    # Image link
    path = '//*[contains(@class, "ad-ui-modal_content")]//img'
    img_link = browser.find_elements_by_xpath(path)[0].get_attribute('src')
    
    return img_link

In [ ]:
def open_close_big_img():
    # Click on Image
    path = '//*[@class="awssld"]'
    browser.find_elements_by_xpath(path)[0].click()
    sleep(0.5)
    # Close Image
    path = '//*[@data-testid="gallery-modal-close-button"]'
    browser.find_elements_by_xpath(path)[0].click()

In [ ]:
def collect_images():
    img_links = []
    try:
        # Images number
        path = '//*[@id="gallerybar"]//div'
        els = browser.find_elements_by_xpath(path)
        img_number = len(els)
        img_links.append(get_img_link())
    except:
        img_number=2
        pass
    sleep(0.5)
    for i in range(img_number-1):
        # Image gallery scroll
        path = '//*[@id="gallerybar"]/div[{}]/img'.format(i+1)
        try:
            browser.find_elements_by_xpath(path)[0].click()
            sleep(0.5)
            link = get_img_link()
        except:
            continue
        
        if link not in img_links:
            img_links.append(link)
    return img_links

In [ ]:
def get_page_data():
    # Images
    images = collect_images()
    images = ', '.join(images)
    
    # Title
    path = '//*[@id="product-information"]/h1'
    title = browser.find_elements_by_xpath(path)[0].text
    
    # Price
    path = '//*[contains(@class, "price_main")]'
    price = browser.find_elements_by_xpath(path)[0].text
    
    # SKU
    path = '//*[contains(@class, "articleNumber")]'
    sku = browser.find_elements_by_xpath(path)[0].text
    
    # Minor Data
    path = '//*[contains(@class, "Component-unordered_list")]'
    minor_data = browser.find_elements_by_xpath(path)[0].text
    
    # Show More Data
    actions = ActionChains(browser)
    for _ in range(6):
        actions.send_keys(Keys.PAGE_DOWN)
        actions.perform()
        sleep(0.1)
    sleep(1)
    for _ in range(4):
        actions.send_keys(Keys.PAGE_UP)
        actions.perform()
        sleep(0.1)

    sleep(2)
    try:
        path = '//*[@data-testid="attributes-show-more"]/button'
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, path)))
        sleep(1)
        element.click()
    except:
        pass
    
    # Description
    try:
        path = '//*[@id="details"]'
        descr = browser.find_elements_by_xpath(path)[0].text
    except:
        descr=np.nan
        pass
    
    sleep(2)
    # Main Data
    cols = []
    values = []
    path = '//*[@id="attribute"]//li' 
    for el in browser.find_elements_by_xpath(path):
        col = el.text.split('\n')[0]
        value = el.text.split('\n')[-1]
        cols.append(col)
        values.append(value)
    page_data = pd.DataFrame([values], columns=cols)
    page_data['Title'] = title
    page_data['Price'] = price
    page_data['SKU'] = sku
    page_data['Minor Data'] = minor_data
    page_data['Description'] = descr
    page_data['URL'] = browser.current_url
    page_data['Images'] = images
    return page_data

## Getting links

In [ ]:
browser = start_browsing()

In [ ]:
# Support functions to parse initial set of links
def get_number_of_pages():
    path = '//*[@class="paging-indicator ng-binding"]'
    pages = browser.find_elements_by_xpath(path)[0].text
    if pages == '':
        pages=2
    pages = int(pages.split('von ')[-1])
    return pages 

def paginate():
    # Pagination
    path = '//*[@class="paging-btn right"]'
    browser.find_elements_by_xpath(path)[0].click()

def populate_section_links():
    path = '//*[@id="article-list"]//a'
    page_links_raw = browser.find_elements_by_xpath(path)
    page_links=[]
    for url in page_links_raw:
        link = url.get_attribute('href')
        if link not in page_links and 'artikel' in link and 'ratings' not in link  and 'artikelvergleich' not in link:
            page_links.append(link)

    section_links.extend(page_links)

In [ ]:
section_links = ['https://www.hornbach.de/shop/Wandverkleidung-Deckenverkleidung/S13928/artikelliste.html',
                'https://www.hornbach.de/shop/Modellbau/S20347/artikelliste.html',
                'https://www.hornbach.de/shop/Moebelbau/S1941/artikelliste.html',
                'https://www.hornbach.de/shop/Bauholz/S1940/artikelliste.html',
                'https://www.hornbach.de/shop/OSB-Platten-Verlegeplatten/S610/artikelliste.html',
                'https://www.hornbach.de/shop/Gartenbau|-Landschaftsbau/Terrassenbau/Terrassendielen/Holz-Terrassendielen/S840/artikelliste.html',
                'https://www.hornbach.de/shop/Gartenbau-Landschaftsbau/Terrassenbau/Terrassendielen/Unterkonstruktionen/S843/artikelliste.html']

In [ ]:
all_links = []
for n, url in enumerate(section_links[:2]):
    browser.get(url)
    pages = get_number_of_pages()
    print(f"\nSection: {n+1} | Number of pages: {pages} | {url}")
    
    section_links = []
    for i in range(0, pages-1):
        sleep(2)
        if i > 0:
            paginate()
        sleep(2)
        populate_section_links()
        print(f"Page: {i+1} | Total links per section: {len(section_links)}")
    
    all_links.extend(section_links)
    print('Total Links:', len(all_links))

### Adding optional links

In [ ]:
%%time
for i, url in enumerate(all_links[:]):
    browser.get(url)
    temp_page_links = get_option_links()
    collect_links(process_temp_links(temp_page_links))
    print(i, f"Total links: {len(all_links)}")

# Getting products data

In [ ]:
df = pd.DataFrame()
for i, url in enumerate(all_links[:]):
    browser.get(url)
    try:
        page_df = get_page_data()
    except:
        print('Error')
        continue
    page_df = page_df.loc[:,~page_df.columns.duplicated()]
    df = pd.concat([df, page_df], ignore_index=True)
    
    if i%10==0:
        print(i, df.shape)
        display(page_df)